In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Dense
import numpy as np
import matplotlib.pyplot as plt
import os
from tqdm import tqdm
tf.keras.backend.set_floatx('float64')
from misc import *
from collections import deque
from nets import Q1, Actor, Q2
import random


def real_training(run_id, lr = 10**-2,
 states_wasted=10**3, batch_size=64, epochs=10, buffer_size=10**3, predict_q_table=True):

    q1=Q1()
    q2=Q2()
    actor = Actor()

    optimizer_critic = tf.keras.optimizers.Adam(lr = lr)
    optimizer_actor = tf.keras.optimizers.Adam(lr = lr)

    pt=[]
    rt=[]
    rts = []
    loss_ev = []
    history = []
    history_would_have_done=[]

    betas_test = np.arange(-1,1,.01)
    optimal = max(ps_maxlik(betas_test))
    buffer = ReplayBufferKennedy(buffer_size=buffer_size)

    for episode in tqdm(range(states_wasted)):

        phase = np.random.choice([-1,1])

        beta_would_do = np.squeeze(actor.give_action().numpy())
        g=[]
        for n1 in [0,1]:
            g.append([-1,1][np.argmax(np.squeeze(q2( np.expand_dims(np.array([[beta_would_do, n1]]) ,axis=1 )).numpy()))])
            
        beta = beta_would_do + np.random.uniform(-.25,.25)
        history.append(beta)
        history_would_have_done.append(beta_would_do)

        outcome = give_outcome(phase*0.4, beta)
        preds_guess = np.squeeze(q2( np.expand_dims(np.array([[beta, outcome]]) ,axis=1 )).numpy())
        guess = [-1,1][np.argmax(preds_guess)]

        pt.append(ps(beta_would_do,g))
        if guess == phase:
            reward=1
        else:
            reward = 0

        rt.append(reward)
        rts.append(np.sum(rt))
        buffer.add(beta, outcome, guess, reward)

        actions_did, outcome, guessed, rewarded = buffer.sample(batch_size)

        if episode >0:
            with tf.GradientTape() as tape:
                tape.watch(q1.trainable_variables)
                predictions = q1(np.expand_dims(np.array(actions_did),axis=1))
                loss_sum = tf.keras.losses.MSE(np.expand_dims(np.array(rewards),axis=1),predictions)
                loss = tf.reduce_mean(loss_sum)
                grads = tape.gradient(loss, q1.trainable_variables)
                optimizer_critic.apply_gradients(zip(grads, q1.trainable_variables))
                loss_ev.append(np.squeeze(loss.numpy()))

        return

check_folder("matese")

for k in range(1):
    run_id=record()
    number_run = "run_"+str(run_id)
    real_training(run_id=number_run, lr=1e-4,
     states_wasted=10**1, batch_size=500, buffer_size=2*10**3)
os.chdir("..")



  0%|          | 0/10 [00:00<?, ?it/s]


ValueError: too many values to unpack (expected 2)